In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../grams/sg_de.csv', names=['frequency', 'word1', 'word2']) # results of skip-grams and 2-grams

In [3]:
# list of ~94k US babynames to remove (hello, john), (james, bond)
names = pd.read_csv("../names.csv", names=['name'])['name'].tolist()

In [4]:
german_filter = ['ich', 'du', 'sie', 'mir', 'wir', 'ihr', 'uns', 'euch', 'die', 'das', 'der', 'den', 'er', 'und', '!', '/', 'ein', 'eine', 'einer', 'einem', 'mein', 'dein', 'nicht', 'wie', 'wo', 'wann', 'was']

In [5]:
dutch_filter = ['ik', 'jij', 'hij', 'zij', 'wij', 'we', 'jullie', 'hun', 'mij', 'me', 'mijn', 'haar', 'hem', '!', '/', 'een', 'de', 'het', 'wie', 'wat', 'waar', 'hoe', 'niet']

In [6]:
l = names + german_filter

In [7]:
df = \
    df[ \
    ~df.word1.isin(l) & \
    ~df.word2.isin(l) & \
    ~df.word1.str.isnumeric() & \
    ~df.word2.str.isnumeric() &\
    ~(df.word1 == df.word2)
    ]

In [8]:
df.count()['frequency']

277582

In [9]:
wc_de = pd.read_csv("../word_counts/wc_de.csv", lineterminator='\n', names=['word', 'word_frequency'])

In [10]:
df = df.merge(wc_de, left_on='word1', right_on='word', how='left') \
.merge(wc_de, left_on='word2', right_on='word', how='left') \
.drop(['word_x', 'word_y'], axis=1) \
.rename(index=str, columns={"word_frequency_x": "word_1_frequency", "word_frequency_y": "word_2_frequency"})

In [11]:
df['normalised_frequency'] = df['frequency'] / (df['word_1_frequency']+df['word_2_frequency']) # divide skipgram freq. by wordcount freq.

In [12]:
df = df.sort_values(by='frequency', ascending=False)

In [13]:
df

,frequency,word1,word2,word_1_frequency,word_2_frequency,normalised_frequency
11387,607621,es,ist,3196654,4236656,0.081743
84440,306065,es,zu,3196654,1991560,0.058992
77574,260714,ist,es,4236656,3196654,0.035074
151261,256849,um,zu,583880,1991560,0.099730
170600,209711,ist,zu,4236656,1991560,0.033671
90198,182353,tut,leid,240465,194850,0.418899
1461,161627,wenn,es,971053,3196654,0.038781
67857,161274,es,war,3196654,988962,0.038531
80309,145130,es,gibt,3196654,277759,0.041771
4258,142900,dass,es,1195857,3196654,0.032533
